<a href="https://colab.research.google.com/github/Redwolf0023/AutoMLvsBayesian/blob/main/Avocado_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download and install the dependencies for H2O
!sudo apt-get install swig -y
!pip install Cython numpy
!pip install scikit-learn
!apt-get install default-jre
!java -version
!pip install h2o

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 16 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,362 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline


In [ ]:
# import the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h2o
import seaborn as sns
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
# input the file as a pandas dataset
df = pd.read_csv('avocado.csv')

In [ ]:
df.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [ ]:
df.drop('Unnamed: 0', axis=1,inplace=True)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [ ]:
# convert the date column to datetime format
df.Date = pd.to_datetime(df.Date)
df = df.assign(
    order_month = df.Date.dt.month,
    order_week_of_the_year =  df.Date.dt.weekofyear,
    order_year = df.Date.dt.year
)
df.shape

<ipython-input-6-031de4eb7a40>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  order_week_of_the_year =  df.Date.dt.weekofyear,


(18249, 16)

In [ ]:
df = df.drop('Date',axis = 1)
df.head()

KeyError: ignored

In [ ]:
from sklearn.preprocessing import LabelEncoder
# List of columns to be label encoded
columns_to_encode = ['type', 'region']

# Initialize the LabelEncoder
encoder = LabelEncoder()

# Create a new DataFrame to store the encoded values
encoded_df = df.copy()

# Apply LabelEncoder to each column and replace the original values
for col in columns_to_encode:
    encoded_df[col] = encoder.fit_transform(df[col])


In [ ]:
df1= encoded_df.copy()
df1.head()

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,order_month,order_week_of_the_year,order_year
0,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,0,2015,0,12,52,2015
1,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,0,2015,0,12,51,2015
2,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,0,2015,0,12,50,2015
3,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,0,2015,0,12,49,2015
4,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,0,2015,0,11,48,2015


In [ ]:
df1['AveragePrice'] = df1['AveragePrice'].astype(int)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   AveragePrice            18249 non-null  int64  
 1   Total Volume            18249 non-null  float64
 2   4046                    18249 non-null  float64
 3   4225                    18249 non-null  float64
 4   4770                    18249 non-null  float64
 5   Total Bags              18249 non-null  float64
 6   Small Bags              18249 non-null  float64
 7   Large Bags              18249 non-null  float64
 8   XLarge Bags             18249 non-null  float64
 9   type                    18249 non-null  int64  
 10  year                    18249 non-null  int64  
 11  region                  18249 non-null  int64  
 12  order_month             18249 non-null  int64  
 13  order_week_of_the_year  18249 non-null  int64  
 14  order_year              18249 non-null

In [ ]:
# Initialize the cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20" 2023-07-18; OpenJDK Runtime Environment (build 11.0.20+8-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20+8-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpczngwhqi
  JVM stdout: /tmp/tmpczngwhqi/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpczngwhqi/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_unknownUser_jyja18
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
# parse the dataset
data = h2o.H2OFrame(df1)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
target_col = data.columns[0]
feature_col = data.columns[1:]
data[target_col] = data[target_col].asfactor()

In [ ]:
# split the dataset into train and test sets
train, test, validation = data.split_frame(ratios=[0.8, 0.15], seed = 12)

In [ ]:
# train the H2O model
model = H2OAutoML(max_models=10, seed = 1234)
model.train(x = feature_col, y = target_col, training_frame=train)

AutoML progress: |████ (cancelled)


H2OJobCancelled: ignored

In [ ]:
#import the libraries used for Gaussian Process
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,ConstantKernel, DotProduct, Matern,ExpSineSquared,WhiteKernel,RationalQuadratic
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc, log_loss, r2_score,mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.metrics import roc_auc_score as ras

In [ ]:
x = df1.drop('AveragePrice', axis=1).values
y = df1['AveragePrice'].values

In [ ]:
# scale the feature column
from sklearn import preprocessing
SS = preprocessing.StandardScaler()
x = SS.fit_transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((14599, 14), (3650, 14), (14599,), (3650,))

In [ ]:
# Initialize the Gaussian Process Regressor using RationalQuadratic kernel
kernel = 1.0* RationalQuadratic(length_scale=1.0, alpha=1.0)
gp = GaussianProcessRegressor(kernel=kernel, alpha=5e-9, optimizer='fmin_l_bfgs_b',
                                n_restarts_optimizer=0, normalize_y=False, copy_X_train=True,
                                random_state=20)

# Train the Gaussian Process Regressor
gp.fit(x_train, y_train)

# Make predictions on the test set
predictions = gp.predict(x_test)

In [ ]:
# evaluate the model
mse = mean_squared_error(y_test, predictions)

print(f"Mean Squared Error: {mse}")
r2 = r2_score(y_test, predictions)

# Print the calculated metrics

print(f"R2 Score: {r2}")